## Veiew DB in panadas

In [3]:
import sqlite3
import pandas as pd

dat = sqlite3.connect('option_strategies.db')
query = dat.execute("SELECT * From option_strategies")
cols = [column[0] for column in query.description]
results= pd.DataFrame.from_records(data = query.fetchall(), columns = cols)
results

,id,scrape_date,strategy_type,tab_name,ticker,trigger_price,strike_price,strike_buy,strike_sell,estimated_premium,last_price_when_checked,timestamp_of_price_when_last_checked,item_id,options_expiry_date,date_info,timestamp_of_trigger,strategy_status,price_when_triggered,price_when_order_placed
0,1,2025-03-22T11:03:01.806121,Bear Call,Mild Risk 95-97% accuracy > shorter expiry,MSFT,401.75,sell 420.0 - buy 430.0,430.0,420.0,75.0,386.84,2025-03-22T11:05:47.201412,1,2025-04-04,March,None,None,None,None
1,2,2025-03-22T11:03:02.235089,Bear Call,Mild Risk 95-97% accuracy > shorter expiry,UNH,523.96,sell 555.0 - buy 565.0,565.0,555.0,88.0,511.30,2025-03-22T11:05:47.216251,2,2025-04-04,March,None,None,None,None
2,3,2025-03-22T11:03:09.999565,Bear Call,Minimal Risk 97-99% accuracy > shorter expiry,None,None,None,0.0,0.0,None,NaN,None,None,2025-04-04,March,None,None,None,None
3,4,2025-03-22T11:03:17.220384,Bear Call,Mild Risk 95-97% accuracy > longer expiry,UNH,523.19,sell 560.0 - buy 570.0,570.0,560.0,170.0,511.30,2025-03-22T11:05:47.186487,1,2025-04-11,March,None,None,None,None
4,5,2025-03-22T11:03:24.602577,Bear Call,Minimal Risk 97-99% accuracy > longer expiry,None,None,None,0.0,0.0,None,NaN,None,None,2025-04-11,March,None,None,None,None
5,6,2025-03-22T11:03:44.403379,Bull Put,Mild Risk 95-97% accuracy > shorter expiry,MSFT,378.41,sell 360.0 - buy 350.0,350.0,360.0,109.0,386.84,2025-03-22T11:05:47.244864,1,2025-04-04,March,None,None,None,None
6,7,2025-03-22T11:03:44.771654,Bull Put,Mild Risk 95-97% accuracy > shorter expiry,WMT,83.77,sell 81.0 - buy 75.0,75.0,81.0,71.0,85.57,2025-03-22T11:05:47.256696,2,2025-04-04,March,None,None,None,None
7,8,2025-03-22T11:03:51.630693,Bull Put,Minimal Risk 97-99% accuracy > shorter expiry,None,None,None,0.0,0.0,None,NaN,None,None,2025-04-04,March,None,None,None,None
8,9,2025-03-22T11:03:58.896353,Bull Put,Mild Risk 95-97% accuracy > longer expiry,NKE,63.95,sell 62.0 - buy 50.0,50.0,62.0,105.0,71.86,2025-03-22T11:05:47.224855,1,2025-04-11,March,None,None,None,None
9,10,2025-03-22T11:03:59.315041,Bull Put,Mild Risk 95-97% accuracy > longer expiry,TQQQ,56.23,sell 53.0 - buy 45.0,45.0,53.0,168.0,61.60,2025-03-22T11:05:47.235999,2,2025-04-11,March,None,None,None,None


In [18]:
results[~results['timestamp_of_trigger'].isnull()]

,id,scrape_date,strategy_type,tab_name,ticker,trigger_price,strike_price,strike_buy,strike_sell,estimated_premium,item_id,options_expiry_date,date_info,timestamp_of_trigger,strategy_status,price_when_triggered,premium_received
8,9,2025-03-10T08:41:21.213768,Bull Put,Mild Risk 95-97% accuracy > shorter expiry,WMT,89.68,sell 86.0 - buy 75.0,75.0,86.0,58.0,4,2025-03-21,March,2025-03-10T14:51:27.476449,premium too low,87.97,NaN
12,13,2025-03-10T08:41:28.351534,Bull Put,Mild Risk 95-97% accuracy > longer expiry,META,589.67,sell 540.0 - buy 530.0,530.0,540.0,97.0,2,2025-03-28,March,2025-03-10T14:57:11.292093,premium too low,589.12,NaN
14,15,2025-03-10T08:41:28.538762,Bull Put,Mild Risk 95-97% accuracy > longer expiry,WMT,89.29,sell 86.0 - buy 80.0,80.0,86.0,70.0,4,2025-03-28,March,2025-03-10T14:51:25.356915,premium too low,87.97,NaN
16,17,2025-03-10T08:41:32.218233,Bull Put,Minimal Risk 97-99% accuracy > longer expiry,QQQ,490,sell 460.0 - buy 450.0,450.0,460.0,273.0,2,2025-03-28,March,2025-03-10T08:46:27.463750,premium too low,488.20,NaN
17,18,2025-03-10T08:41:32.354756,Bull Put,Minimal Risk 97-99% accuracy > longer expiry,TQQQ,62.25,sell 60.0 - buy 50.0,50.0,60.0,210.0,3,2025-03-28,March,2025-03-10T14:33:41.525971,premium too low,61.69,NaN
46,47,2025-03-12T06:02:01.555882,Bull Put,Minimal Risk 97-99% accuracy > shorter expiry,JNJ,163.62,sell 157.5 - buy 148.0,148.0,157.5,72.0,1,2025-03-28,March,2025-03-12T21:19:42.678459,order placed,162.70,60.0


## Delete ALL rows in SQLite database

In [32]:

import sqlite3

conn = sqlite3.connect('option_strategies.db')
c = conn.cursor()

# Delete all rows from table
c.execute('DELETE FROM option_strategies;',)

print('We have deleted', c.rowcount, 'records from the table.')

# Commit the changes to db			
conn.commit()
# Close the connection
conn.close()

We have deleted 18 records from the table.


## Delete table

In [1]:

# importing sqlite module 
import sqlite3 
  
# create connection to the  
# database geek 
connection = sqlite3.connect('option_strategies.db') 
  
# drop table 
connection.execute("DROP TABLE option_strategies") 
  
print("data dropped successfully") 
  
# close the connection 
connection.close() 

data dropped successfully


## Update Row

In [37]:
import sqlite3

sql  = 'UPDATE option_strategies SET trigger_price=? WHERE id = ?'

try:
    with sqlite3.connect('option_strategies.db') as conn:
        cur = conn.cursor()
        cur.execute(sql, (490, 17))
        conn.commit()
except sqlite3.OperationalError as e:
    print(e)